# Import Useful Modules 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time import gmtime, strftime
import time
import datetime
from collections import Counter
import pickle
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Preprocessing
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Embedder
from gensim.models import FastText

# Classifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier as GBC
from sklearn.linear_model import LogisticRegression as LR
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.grid_search import GridSearchCV as GS

# import keras
# import tensorflow
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.utils import to_categorical

from preprocessing_pipeline import preprocessing

import tqdm

D:\Program\Anaconda\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
D:\Program\Anaconda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
D:\Program\Anaconda\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
D:\Program\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversi

In [4]:
#Determine Model's Save Location

#version=
version="Fixed"

In [5]:
if(not(os.path.exists("model/{}".format(version)))):
    os.makedirs("model/{}".format(version))

# ---------------------------------------------------------------------------------------------------------------

# Read Data To Pandas Dataframe

In [6]:
def read_product_title_data():
    product_title_only=pd.read_csv('data/query.csv',header=None)
    product_title_only["Product Title"]=product_title_only[1]
    product_title_only=product_title_only[["Product Title"]]
    product_title_only.dropna(inplace=True,axis=0)
    
    return product_title_only

In [7]:
data_for_embedding=read_product_title_data()

In [8]:
data_for_embedding.head()

,Product Title
0,gear+camping
1,Lightstick+EXO
2,vivo+v7
3,subwoofer+kolong+aktif
4,Hotpants+anak


# ---------------------------------------------------------------------------------------------------------------

# Construct Word Embedder (Using fasttext)

In [9]:
preprocessor=preprocessing(None,None)

In [11]:
product_title=[preprocessor.tokenize(value) for value in tqdm.tqdm(data_for_embedding["Product Title"])]


 17%|████████████▌                                                           | 426140/2440682 [00:07<00:34, 58427.39it/s]


 37%|██████████████████████████▎                                             | 891795/2440682 [00:14<00:25, 61880.67it/s]


 55%|███████████████████████████████████████▍                               | 1354489/2440682 [00:21<00:17, 62352.34it/s]


 75%|█████████████████████████████████████████████████████                  | 1824916/2440682 [00:28<00:09, 63282.05it/s]


 94%|██████████████████████████████████████████████████████████████████▊    | 2297103/2440682 [00:36<00:02, 63717.88it/s]


100%|███████████████████████████████████████████████████████████████████████| 2440682/2440682 [00:38<00:00, 64061.90it/s]

In [16]:
new_product_title=[]
for sentence in tqdm.tqdm(product_title):
    new_product_title.append([word for word in sentence if result[word]>=50 and len(word)>2])
new_product_title=[title for title in tqdm.tqdm(new_product_title) if len(title)>=2]


100%|█████████████████████████████████████████████████████████████████████| 2440682/2440682 [00:01<00:00, 1834357.18it/s]

In [17]:
result=preprocessor.word_count(product_title)
result2=preprocessor.word_count(new_product_title)

1/1
1/1


In [20]:
len(result2)/len(result)

0.12089641095424392

In [22]:
len(new_product_title)/len(product_title)

0.7202703998308669

#### Train Embedding Model

In [38]:
EMBEDDING_DIMENSION=100
EMBEDDING_EPOCH=100
EMBEDDING_WINDOW=7
MIN_COUNT=10
SEED=2918342

In [39]:
# membuat model word embedding
print("Start Time : {}\n".format(str(datetime.datetime.now())))
word_embedder = FastText(new_product_title[:100000], size=EMBEDDING_DIMENSION, window=EMBEDDING_WINDOW, min_count=MIN_COUNT, workers=16, sg=1, seed=SEED, min_n=5, iter=EMBEDDING_EPOCH)
print("End Time : {}".format(str(datetime.datetime.now())))

Start Time : 2018-08-08 17:50:29.146899

End Time : 2018-08-08 17:52:32.605620


#### Save Model

In [32]:
# #save model
# pickle.dump(word_embedder, open("model/{}/word_embedder_{}.pickle".format(version,EMBEDDING_DIMENSION), 'wb'))

# ---------------------------------------------------------------------------------------------------------------

# Word Embedding Evaluation

#### Fictional Character

In [40]:
word_embedder.wv.most_similar("mobil")

[('mobilan', 0.6665498614311218),
 ('brooks', 0.6144686937332153),
 ('mobilio', 0.5463486909866333),
 ('serep', 0.5144010186195374),
 ('maxx', 0.5107284784317017),
 ('freed', 0.48475661873817444),
 ('plafon', 0.4757026135921478),
 ('rubicon', 0.4727756083011627),
 ('organizer', 0.46525830030441284),
 ('tukang', 0.4617008566856384)]

#### Computer Brand

In [41]:
word_embedder.wv.most_similar("intel")

[('quad', 0.7961403131484985),
 ('procesor', 0.7942219972610474),
 ('amd', 0.7509782314300537),
 ('built', 0.7214125394821167),
 ('prosesor', 0.7135754227638245),
 ('processor', 0.7010242938995361),
 ('ryzen', 0.684984564781189),
 ('intex', 0.6610298156738281),
 ('vaio', 0.6570157408714294),
 ('lga', 0.6483675837516785)]

In [42]:
word_embedder.wv.most_similar("acer")

[('aspire', 0.6963398456573486),
 ('netbook', 0.6534676551818848),
 ('inspiron', 0.6520154476165771),
 ('toshiba', 0.6191145777702332),
 ('himax', 0.616962730884552),
 ('neca', 0.6091619729995728),
 ('ideapad', 0.596017599105835),
 ('ryzen', 0.5898139476776123),
 ('axioo', 0.5845321416854858),
 ('nitro', 0.5814634561538696)]

### Automotive Brand

In [43]:
word_embedder.wv.most_similar("honda")

[('jerigen', 0.5491578578948975),
 ('primus', 0.5002319812774658),
 ('juken', 0.4936077296733856),
 ('ferio', 0.4866533875465393),
 ('fiesta', 0.4853607714176178),
 ('camry', 0.48482197523117065),
 ('tengkorak', 0.4839434027671814),
 ('vega', 0.4784013032913208),
 ('repairkit', 0.4767449200153351),
 ('shad', 0.4748554825782776)]

In [44]:
word_embedder.wv.most_similar("yamaha")

[('midi', 0.5278173685073853),
 ('arai', 0.5064706802368164),
 ('parker', 0.49939867854118347),
 ('xabre', 0.49812889099121094),
 ('kumpulan', 0.4963878393173218),
 ('lagu', 0.4847766160964966),
 ('esp', 0.4830368459224701),
 ('psr', 0.4811265468597412),
 ('maru', 0.47903478145599365),
 ('cld', 0.47867780923843384)]

In [45]:
word_embedder.wv.most_similar("samsung")

[('galaxy', 0.6051572561264038),
 ('mito', 0.552219033241272),
 ('gerigi', 0.5494296550750732),
 ('note', 0.5469122529029846),
 ('lenovo', 0.5417517423629761),
 ('cumi', 0.5330456495285034),
 ('meizu', 0.5315901041030884),
 ('stylus', 0.5241528749465942),
 ('sony', 0.5186532735824585),
 ('himax', 0.5179975628852844)]

# ---------------------------------------------------------------------------------------------------------------